In [1]:
import random
import pprint
from operator import itemgetter
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from mayavi import mlab
import functools







In [2]:
N=10
data_points = []
rng = 100
partition_count=3
# for i in range(N):
#     for j in range(N):
#         for k in range(N):
#             data_points.append([i,j,k])


for i in range(N):
    for j in range(N):
        # x = random.uniform(-1,1)
        x = random.randint(0,rng)

        # y = random.uniform(-1,1)
        y = random.randint(0,rng)
        z = random.randint(0,rng)
        data_points.append([x,y,z])

In [3]:
"""
unsigned int x = (m_uiX ^ other.m_uiX);
unsigned int y = (m_uiY ^ other.m_uiY);
unsigned int z = (m_uiZ ^ other.m_uiZ);

//Default pref: z > y > x.
unsigned int maxC = z;
unsigned int yOrx = y;
if (yOrx < x) 
{
    if ((x ^ yOrx) >= yOrx) {
        yOrx = x;
    }
}
if (maxC < yOrx) {
    if ((maxC ^ yOrx) >= maxC) {
        maxC = yOrx;
    }
}

if (maxC == z) {
    return (m_uiZ < other.m_uiZ); 
} else if (maxC == y) {
    return (m_uiY < other.m_uiY); 
} else {
    return (m_uiX < other.m_uiX); 

}
// -- original Morton


"""

def morton_compare(p1, p2):
    # p1 = p1_data["value"]
    # p2 = p2_data["value"]
    temp_x = p1[0] ^ p2[0]
    temp_y = p1[1] ^ p2[1]
    temp_z = p1[2] ^ p2[2]

    maxC = temp_z
    yOrx = temp_y

    if (yOrx < temp_x):
        if ((temp_x ^ yOrx) >= yOrx):
            yOrx = x

    if (maxC < yOrx):
        if ((maxC ^ yOrx) >= maxC):
            maxC = yOrx
        
    if (maxC == temp_z):
        return -1 if p1[2] < p2[2] else 1
    elif (maxC == temp_y):
        return -1 if p1[1] < p2[1] else 1
    else:
        return -1 if p1[0] < p2[0] else 1

data_points_morton_ordered = sorted(data_points, key=functools.cmp_to_key(morton_compare))

figsize=15
G = nx.Graph()
G.add_node(0)
pos = {}
pos[0] = data_points_morton_ordered[0]
for i in range(1,len(data_points)):
    G.add_node(i)
    G.add_edge(i-1,i)
    pos[i] = data_points_morton_ordered[i]
# plt.figure(figsize=(figsize,figsize))
# nx.draw_networkx(G, pos)

In [4]:
def cluster_to_color(ci):
    random.seed(ci*ci)
    return (random.randint(0,255),random.randint(0,255),random.randint(0,255),255)

In [5]:
partition_size = (len(data_points))//partition_count
large_partition_count = len(data_points) % partition_count
morton_only_partition_labels = [-1 for _ in range(len(data_points))]

centers = []

for partition_idx in range(partition_count):

    if partition_idx < large_partition_count:
        size = partition_size+1
        offset = partition_idx*(partition_size+1)
    else:
        size = partition_size
        offset = large_partition_count*(partition_size+1) + (partition_idx-large_partition_count)*partition_size
    for i in range(offset, offset+size):
        morton_only_partition_labels[i] = partition_idx
        # print(morton_order[i])
print(morton_only_partition_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [6]:
mlab.options.backend = 'envisage'
# s = mlab.test_plot3d()
# mlab.figure()
xyz = np.array(data_points_morton_ordered)
# scalar colors
scalars = np.array(list(G.nodes()))

# mlab.figure()
pts = mlab.points3d(
    xyz[:, 0],
    xyz[:, 1],
    xyz[:, 2],
    scalars,
    scale_factor=0.09,
    scale_mode="none",
    colormap="Blues",
    resolution=20,
)

clrs = [cluster_to_color(label) for label in morton_only_partition_labels]
pts.module_manager.scalar_lut_manager.lut.number_of_colors = partition_count
pts.module_manager.scalar_lut_manager.lut.table = clrs

# edges_adjusted_for_idx = [(element_to_idx[u],element_to_idx[v]) for u,v in G.edges()]
pts.mlab_source.dataset.lines = np.array(G.edges())
tube = mlab.pipeline.tube(pts, tube_radius=0.02)
mlab.pipeline.surface(tube, color=(0.8, 0.8, 0.8))

mlab.orientation_axes()
mlab.show()

/home/budvin/research/Partitioning/paralab-partition/.venv/lib/python3.10/site-packages/pyface/ui/qt/workbench/split_tab_widget.py:125: UserWarning: Attempting to restore to a non-empty widget.
  warnings.warn("Attempting to restore to a non-empty widget.")


FileNotFoundError: [Errno 2] No such file or directory: '/home/budvin/.enthought/mayavi_e3/preferences.ini'